# Weekly Project 2

## Robot Tracking
For this project you are given a video of some mobile robots (Robots.mp4). The task is now to track only the robots that are moving. Try to use both sparse and dense optical flow and compare the results.

For sparse optical flow, draw the tracked keypoints onto each frame and try to show the frames fast enough, such that it looks like a video.

For dense optical flow, represent the movement in any way you see fitting. For example by making a new image with the colors of each pixel representing the movement.

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

Loading the video and getting the frames for test

In [2]:
path = "C:/Users/shaia/Documents/Opgaveregning/AutoSys/4. semester/Perception for autonome systemer/Projekt 2/"
video_name = "Robots.mp4"

cap = cv2.VideoCapture(path + video_name)
success1, frame1 = cap.read()
cap.set(cv2.CAP_PROP_POS_FRAMES, 2)
success2, frame2 = cap.read()

Tracking the robot with sparse optical flow

In [3]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
success, frame = cap.read()
frames = [frame]
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cap.set(cv2.CAP_PROP_POS_FRAMES, 2)

while True:
    success, frame = cap.read()
    if not success:
        break
    next_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    feat1 = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.3, minDistance=7)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(gray, next_gray, feat1, None)

    for i in range(len(feat1)):
        f10=int(feat1[i][0][0])
        f11=int(feat1[i][0][1])
        f20=int(feat2[i][0][0])
        f21=int(feat2[i][0][1])
        if np.abs(f10 - f20) < 2 and np.abs(f11 - f21) < 2:
            continue
        cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)
        cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)

    gray = next_gray.copy()
    frames.append(frame)

Watch the video for sparse optical flow

In [4]:
for image in frames:
    cv2.namedWindow('Frames', cv2.WINDOW_NORMAL)
    cv2.imshow('Frames', image)
    cv2.waitKey(20)
cv2.destroyAllWindows()

Tracking the robot with dense optical flow

In [5]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
success, frame = cap.read()
frames2 = [frame]
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    success, frame = cap.read()
    if not success:
        break
    next_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.5, 0)
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])
    hsv = np.zeros_like(frame)
    hsv[:, :, 0] = ang * 180 / np.pi / 2
    hsv[:, :, 1] = np.ones_like(frame[:, :, 0]) * 255
    hsv[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    gray = next_gray.copy()
    frames2.append(frame + bgr)

Watch the video for dense optical flow

In [6]:
for image in frames2:
    cv2.namedWindow('Frames', cv2.WINDOW_NORMAL)
    cv2.imshow('Frames', image)
    cv2.waitKey(20)
cv2.destroyAllWindows()

Save the video

In [7]:
height, width, _ = frames[0].shape

out = cv2.VideoWriter("Sparse optical flow.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
for frame in frames:
    out.write(frame)

out2 = cv2.VideoWriter("Dense optical flow.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
for frame in frames2:
    out2.write(frame)

out.release()
out2.release()
cap.release()